In [2]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox, ttk
import numpy as np
from transformers import MarianMTModel, MarianTokenizer

# Define available languages and their MarianMT model codes
LANGUAGE_CODES = {
    "English": "en",
    "Spanish": "es",
    "German": "de"
}

# MarianMT model mapping for translation pairs
TRANSLATION_MODELS = {
    ("en", "es"): "Helsinki-NLP/opus-mt-en-es",
    ("en", "de"): "Helsinki-NLP/opus-mt-en-de",
    ("es", "en"): "Helsinki-NLP/opus-mt-es-en",
    ("de", "en"): "Helsinki-NLP/opus-mt-de-en",
    ("es", "de"): "Helsinki-NLP/opus-mt-es-de",
    ("de", "es"): "Helsinki-NLP/opus-mt-de-es",
}

# Load translation model
def load_translation_model(src_lang, tgt_lang):
    model_name = TRANSLATION_MODELS.get((src_lang, tgt_lang))
    if model_name:
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)
        return tokenizer, model
    return None, None

# Function to translate text
def translate_text(text, src_lang, tgt_lang):
    if src_lang == tgt_lang:
        return text  # No need to translate

    tokenizer, model = load_translation_model(src_lang, tgt_lang)
    if not model:
        return text  # Model not available for the selected pair

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**inputs)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

# Function to update UI text based on selected language
def update_language(*args):
    global current_language
    new_language = language_var.get()

    if new_language == current_language:
        return  # Avoid redundant updates

    src_lang = LANGUAGE_CODES[current_language]
    tgt_lang = LANGUAGE_CODES[new_language]
    
    # Translate all UI elements
    labels_texts = {
        label1: "Heart Disease Risk Application",
        label2: "Enter your age:",
        label3: "Enter your cholesterol reading:",
        label4: "Enter your resting heart rate (in beats per minute):",
        label5: "What is your biological gender?",
        label6: "Have you ever smoked?",
        label7: "Have you been diagnosed with exercise-induced angina?",
    }

    for label, text in labels_texts.items():
        translated_text = translate_text(text, src_lang, tgt_lang)
        label.config(text=translated_text)

    # Update button text
    translated_button_text = translate_text("Submit", src_lang, tgt_lang)
    submit_button.config(text=translated_button_text)

    # Set new language
    current_language = new_language

# Function to process responses and give an answer
def process_response():
    try:
        # Get age input and validate
        age = int(age_entry.get())
        if age < 18 or age > 70:
            messagebox.showerror("Invalid input", "This assessment is based on data from 18 to 70 year olds.")
            return
        
        # Get cholesterol input and validate
        cholesterol = int(cholesterol_entry.get())
        if cholesterol > 240:
            messagebox.showerror("Danger for Heart Disease", "Danger of Heart Disease if cholesterol reading is over 240 mg/dL.")
            return
        
        # Get resting heart rate and validate
        resting_hr = int(resting_hr_entry.get())
        if resting_hr > 180: 
            messagebox.showerror("Danger for Heart Disease", "Danger of Heart Disease if resting heart rate reading is over 180 beats per minute.")
            return
        
        # Risk Assessment Function
        def risk_assessment(person_age, person_cholesterol, person_resting_hr):
            user_input = pd.DataFrame({'age': [person_age],'cholesterol': [person_cholesterol], 
                                       'resting_hr': [person_resting_hr]})
            prediction = np.random.rand()
            if prediction > 0.5:
                return 'Heart Disease Attack or Risk'
            else:
                return 'No Heart Disease Attack or Risk'

        # Make a prediction and display result
        result = risk_assessment(age, cholesterol, resting_hr)
        messagebox.showinfo("Result", result)

    except ValueError:
        # Handle non-numeric input
        messagebox.showerror("Invalid Input", "Please enter a valid numeric input.")

# Create the main application window
app = tk.Tk()
app.title("Heart Disease Risk Application")

# Default language
current_language = "English"

# Language Selection Dropdown
tk.Label(app, text="Select Language:").pack()
language_var = tk.StringVar(value="English")
language_dropdown = ttk.Combobox(app, textvariable=language_var, values=list(LANGUAGE_CODES.keys()))
language_dropdown.pack()
language_var.trace_add("write", update_language)

# Header
label1 = tk.Label(app, text="Heart Disease Risk Application")
label1.pack()

# Age Input
label2 = tk.Label(app, text="Enter your age:")
label2.pack()
age_entry = tk.Entry(app)
age_entry.pack()

# Cholesterol input
label3 = tk.Label(app, text="Enter your cholesterol reading:")
label3.pack()
cholesterol_entry = tk.Entry(app)
cholesterol_entry.pack()

# Resting Heart Rate input
label4 = tk.Label(app, text="Enter your resting heart rate (in beats per minute):")
label4.pack()
resting_hr_entry = tk.Entry(app)
resting_hr_entry.pack()

# Question 1: Gender request
label5 = tk.Label(app, text="What is your biological gender?")
label5.pack()
question1_var = tk.StringVar()
tk.Radiobutton(app, text="M", variable=question1_var, value="male").pack()
tk.Radiobutton(app, text="F", variable=question1_var, value="female").pack()

# Question 2: Smoking status
label6 = tk.Label(app, text="Have you ever smoked?")
label6.pack()
question2_var = tk.StringVar()
tk.Radiobutton(app, text="Yes", variable=question2_var, value='yes').pack()
tk.Radiobutton(app, text="No", variable=question2_var, value='no').pack()

# Question 3: Exercise-induced angina status
label7 = tk.Label(app, text="Have you been diagnosed with exercise-induced angina?")
label7.pack()
question3_var = tk.StringVar()
tk.Radiobutton(app, text="Yes", variable=question3_var, value='yes').pack()
tk.Radiobutton(app, text="No", variable=question3_var, value='no').pack()

# Submit Button
submit_button = tk.Button(app, text="Submit", command=process_response)
submit_button.pack()

# Run the application
app.mainloop()


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

c:\Users\adminpl\.conda\envs\dev\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adminpl\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-de-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]